In [5]:
import pickle
import os
import cv2
import numpy as np
import PIL
#PIL.PILLOW_VERSION = PIL.__version__

from src.dataset import VDAODataset
from torch.utils.data import DataLoader
from src import RESULT_DIR
from src.utils import MCC, DIS, conf_mat
import matplotlib.pyplot as plt
import pandas as pd

from scripts.extra.mcDTSR.utils.srmc_outer_repr import srmc_outer_repr

In [6]:
fold = 1
num_frames = 201
dataset = VDAODataset(fold = fold, split = 0, type = 'test',
                      alignment = 'temporal', transform = False)
dd = dataset.align_df
vid_list = dd.test_file.unique()
loader = DataLoader(dataset, num_workers=8, batch_size=1, shuffle=False)
ref_frame, tar_frame, _ , _   = dataset.__getitem__(0)
w, h = ref_frame.shape[1], ref_frame.shape[0]

ref_vid = np.zeros((h, w, 201))
tar_vid = np.zeros((h, w, 201))

In [7]:
for i in range(num_frames):
    ref_frame, tar_frame, _ , _   = dataset.__getitem__(i)
    ref_vid[:, :, i] = cv2.cvtColor(ref_frame, cv2.COLOR_RGB2GRAY)
    tar_vid[:, :, i] = cv2.cvtColor(tar_frame, cv2.COLOR_RGB2GRAY)

In [8]:
ww = 280
hh = 150

params = {'windowSize': (hh, ww),
          'windowPos': ((h-hh)/2, (w-ww)/2),
          'transformType': '4P-HOMOGRAPHY',
          'display': 2,
          'stoppingDelta': 1e-4,
          'rho': 1.01,
          'inner_tol': 1e-4,
          'inner_maxIter': 5000,
          'lambda': 1e2,
          'DISPLAY_EVERY': 10
          }


In [9]:
data = {'prevObj': np.inf,
        'numIterOuter': 0,
        'numIterInner': [],
        'W': {},
        'xi': {},
        'W_1': [],
        'E_1': [],
        'dt_1': [],
        'time': [],
        'rhos': [],
        'rerr': []
        }

In [12]:
# RASL main loop: do robust batch image alignment

maxIter = 100
for k in range(data['numIterOuter']+1, maxIter):
    params['maxIter'] = k
    [Xr, Xt, W, E, xi, data] = srmc_outer_repr(ref_vid, tar_vid, params, data);
    if data['rerr'][k] < params['stoppingDelta']:
        break



info: no T0 given, setting T0 = I


RuntimeError: invalid shape for coordinate array